In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.seasonal import MSTL, DecomposeResult

In [4]:
# ============ loading data =================
df = pd.read_parquet("../data/processed/bordeaux_conso_mwh.parquet")
df.head()

,date,daily_conso_mgw
0,2022-01-01,1153665
1,2022-01-02,1194663
2,2022-01-03,1391133
3,2022-01-04,1432283
4,2022-01-05,1555351


In [5]:
# ========== first checking =============
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             1425 non-null   datetime64[ns]
 1   daily_conso_mgw  1425 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 22.4 KB


In [6]:
# ========== checking missing values ==============
df.isnull().sum()

date               0
daily_conso_mgw    0
dtype: int64

In [7]:
# ============ Checking duplicates values ============= 
df.duplicated().sum()

np.int64(0)

# Eda 

In [8]:
fig = px.line(
    df, 
    x="date", 
    y='daily_conso_mgw', 
    title="Consmmation journalière (MWG) sur Bordeaux", 
    labels={'date': 'Date', 'daily_conso_mgw': 'Consommation (MWg)'}
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Consommation (MWg)',
    template='plotly_white'
)

fig.show()

### 📊 Analyse graphique de la consommation journalière

- **Multi-saisonnalité visible** :  
  - Saisonnalité hebdomadaire (week-end vs semaine).  
  - Saisonnalité annuelle (été vs hiver).  

- **Outliers ponctuels** :  
  - Quelques valeurs extrêmes apparaissent (ex. Mars 2027, Octobre 2022).  
  - Ces points peuvent correspondre à des événements exceptionnels ou des erreurs dans les données.  

- **Trend** :  
  - Aucun trend clair n’est visible à l’œil nu sur la série brute.  
  - La tendance pourrait être masquée par la multi-saisonnalité et le bruit quotidien.  

💡 Remarque : Pour mieux analyser le trend et séparer les composantes saisonnières, une décomposition STL ou un modèle type TBATS/Prophet sera nécessaire.


In [9]:
mstl = MSTL(df['daily_conso_mgw'], periods=[7, 365])
res = mstl.fit()

df['trend'] = res.trend
df['seasonal_7'] = res.seasonal['seasonal_7']
df['seasonal_365'] = res.seasonal['seasonal_365']
df['resid'] = res.resid

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['daily_conso_mgw'], mode='lines', name='Original'))
fig.add_trace(go.Scatter(x=df.index, y=df['trend'], mode='lines', name='Trend'))
fig.add_trace(go.Scatter(x=df.index, y=df['seasonal_7'], mode='lines', name='Seasonal Hebdo'))
fig.add_trace(go.Scatter(x=df.index, y=df['seasonal_365'], mode='lines', name='Seasonal Annuel'))

fig.update_layout(title='Décomposition MSTL de la consommation journalière',
                  xaxis_title='Date',
                  yaxis_title='Consommation (MWg)',
                  template='plotly_white')
fig.show()

# Résidus
fig_resid = go.Figure()
fig_resid.add_trace(go.Scatter(x=df.index, y=df['resid'], mode='markers', name='Résidus'))
fig_resid.update_layout(title='Résidus MSTL (outliers possibles)',
                        xaxis_title='Date',
                        yaxis_title='Résidus',
                        template='plotly_white')
fig_resid.show()

In [10]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# Test Ljung-Box sur les résidus
residus = df['resid'].dropna()
lb_test = acorr_ljungbox(residus, lags=[10, 20, 30], return_df=True)
print(lb_test)

        lb_stat  lb_pvalue
10  2780.829453        0.0
20  2824.880035        0.0
30  2854.468111        0.0


### ⚙️ Justification du choix de modèle

Les modèles classiques comme le **lissage exponentiel (Holt-Winters)** ou **ARIMA/SARIMA** ne gèrent qu’une **saison à la fois**.  

Or, la série présente **au moins deux saisons** :  
- **Hebdomadaire**  
- **Annuelle**  

Ces modèles classiques ne sont donc **pas adaptés** pour capturer correctement toutes les composantes saisonnières.

Pour cette raison, j’ai choisi d’utiliser des modèles capables de gérer plusieurs saisons simultanément :  

- **Prophet** : permet de modéliser plusieurs saisonnalités et d’incorporer des événements ponctuels (jours fériés, événements exceptionnels).  
- **TBATS** : conçu pour gérer plusieurs saisons avec des périodes non entières et des séries longues, tout en capturant la tendance et le niveau de bruit.

---

### 🔜 Transition vers la modélisation

En résumé, après analyse graphique et MSTL :  

- La série montre une **forte multi-saisonnalité**  
- Les résidus **ne sont pas du bruit blanc**, indiquant des motifs restants  
- Les modèles classiques ne suffiraient pas pour la prédiction

> La prochaine étape consiste donc à mettre en place un **modèle multi-saisonnalité** pour générer des forecasts fiables.


In [ ]:

from sklearn.metrics import r2_score
import plotly.graph_objects as go

# --- Préparer les données pour Prophet ---
# df doit avoir colonnes ['date','daily_conso_mgw']
df_fb = df[['date', 'daily_conso_mgw']].copy()
df_fb = df_fb.rename(columns={'date': 'ds', 'daily_conso_mgw': 'y'})

# --- Train / Test split ---
train_size = int(len(df_fb) * 0.8)  # 80% train, 20% test
train = df_fb.iloc[:train_size]
test = df_fb.iloc[train_size:]

# --- Classe Prophet ---
class FbprophetModel:
    def fit(self, data):
        self.data = data.copy()
        self.model = Prophet(
            weekly_seasonality=True,
            daily_seasonality=False,
            yearly_seasonality=True
        )
        self.model.fit(self.data)
    
    def forecast(self, periods, freq):
        self.future = self.model.make_future_dataframe(periods=periods, freq=freq)
        self.df_forecast = self.model.predict(self.future)
        return self.df_forecast
    
    def
        # R² sur les valeurs fournies (ex: test set)
        return r2_score(true_values, self.df_forecast['yhat'][-len(true_values):])

# --- Utilisation ---
model = FbprophetModel()
model.fit(train)
model.forecast(len(test), "D")  # prévoir la période test

# R² sur test
r2 = model.R2(test['y'])
print("R² sur test set:", r2)

11:35:45 - cmdstanpy - INFO - Chain [1] start processing
11:35:45 - cmdstanpy - INFO - Chain [1] done processing


R² sur test set: 0.7470670408076335


In [13]:

start_date = '2025-11-25'
end_date = '2025-11-30'

# Extraire la portion forecast pour cette période
forecast_zoom = model.df_forecast[
    (model.df_forecast['ds'] >= start_date) & (model.df_forecast['ds'] <= end_date)
].set_index('ds')

# Extraire les valeurs réelles si elles existent dans le test
test_zoom = test[(test['ds'] >= start_date) & (test['ds'] <= end_date)]

# --- Plotly ---
fig = go.Figure()

# Valeurs réelles
fig.add_trace(go.Scatter(
    x=test_zoom['ds'],
    y=test_zoom['y'],
    mode='markers+lines',
    name='Actual',
    marker=dict(color='blue')
))

# Valeurs prédites
fig.add_trace(go.Scatter(
    x=forecast_zoom.index,
    y=forecast_zoom['yhat'],
    mode='lines+markers',
    name='Forecast',
    line=dict(color='red')
))

# Intervalle de confiance
fig.add_trace(go.Scatter(
    x=list(forecast_zoom.index) + list(forecast_zoom.index[::-1]),
    y=list(forecast_zoom['yhat_upper']) + list(forecast_zoom['yhat_lower'][::-1]),
    fill='toself',
    fillcolor='rgba(200,200,200,0.3)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='Confidence Interval'
))

fig.update_layout(
    title="Prophet Forecast: 25-30 Nov 2025",
    xaxis_title="Date",
    yaxis_title="Daily Consumption (MWg)",
    template="plotly_white",
    legend=dict(x=0.01, y=0.99)
)

fig.show()

In [ ]:
# --- Préparer le forecast pour Plotly ---
forecast_test = model.df_forecast[['ds','yhat_lower','yhat_upper','yhat']].tail(len(test)).set_index('ds')

# --- Plotly graphique interactif ---
fig = go.Figure()

# Train data
fig.add_trace(go.Scatter(
    x=train['ds'],
    y=train['y'],
    mode='lines',
    name='Train',
    line=dict(color='blue')
))

# Test data
fig.add_trace(go.Scatter(
    x=test['ds'],
    y=test['y'],
    mode='markers+lines',
    name='Test (Actual)',
    marker=dict(color='darkblue')
))

# Forecast
fig.add_trace(go.Scatter(
    x=forecast_test.index,
    y=forecast_test['yhat'],
    mode='lines+markers',
    name='Forecast',
    line=dict(color='red')
))

# Intervalle de confiance
fig.add_trace(go.Scatter(
    x=list(forecast_test.index) + list(forecast_test.index[::-1]),
    y=list(forecast_test['yhat_upper']) + list(forecast_test['yhat_lower'][::-1]),
    fill='toself',
    fillcolor='rgba(200,200,200,0.3)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='Confidence Interval'
))

fig.update_layout(
    title="Prophet Forecast: Train vs Test vs Prediction",
    xaxis_title="Date",
    yaxis_title="Daily Consumption (MWg)",
    template="plotly_white",
    legend=dict(x=0.01, y=0.99)
)

fig.show()
